In [11]:
#check ipynb is running
print("Hello, World!")

Hello, World!


In [12]:
#set working directory
import os
os.chdir("/Users/jovita.brundziene/Python/airflow-de-intro-project-jbru")

In [13]:
#check working directory set to project root to use relative pathways later
os.getcwd()

'/Users/jovita.brundziene/Python/airflow-de-intro-project-jbru'

To do:
- metadata handling
- Go through repo steps
- Include dev/prod environment parameters
- add parameters to config file
- create a docker image
- create a github action to run pipeline automatically
- create unit tests
- modularise code into at least config, functions and run
- Update requirements file and build it into the script
- Requirements lint?
- Nice to have: package it up as a python package?
- add logging

### Extract data from local to S3

In [14]:
import boto3
import os
import logging
from botocore.exceptions import ClientError

# Set up logging configuration
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

def upload_parquet_files_to_s3(bucket_name, local_directory, s3_prefix, dry_run=True):
    """
    Uploads .parquet files from a local directory to an S3 bucket under a specified prefix.

    Parameters:
        bucket_name (str): Name of the S3 bucket.
        local_directory (str): Path to the local directory containing .parquet files.
        s3_prefix (str): Path prefix within the S3 bucket.
        dry_run (bool): If True, simulates the upload without actually uploading files.
    """
    # Create an S3 client using boto3
    s3 = boto3.client('s3')

    # Loop through all files in the specified local directory
    for file in os.listdir(local_directory):
        # Only process files with a .parquet extension
        if file.endswith('.parquet'):
            # Construct the full local file path
            local_path = os.path.join(local_directory, file)
            # Define the S3 object key (i.e., path within the bucket)
            s3_key = f'{s3_prefix}/{file}'

            try:
                # Check if the file already exists in the S3 bucket
                s3.head_object(Bucket=bucket_name, Key=s3_key)
                logging.info(f"File already exists in S3: s3://{bucket_name}/{s3_key} — skipping upload.")
            except ClientError as e:
                # If the error code is 404, the file does not exist — proceed with upload
                if e.response['Error']['Code'] == '404':
                    if dry_run:
                        # Simulate the upload in dry run mode
                        print(f"[DRY RUN] Would upload: {local_path} to s3://{bucket_name}/{s3_key}")
                    else:
                        # Attempt to upload the file to S3
                        try:
                            s3.upload_file(local_path, bucket_name, s3_key)
                            logging.info(f"Successfully uploaded: {local_path} to s3://{bucket_name}/{s3_key}")
                        except Exception as upload_error:
                            logging.error(f"Failed to upload: {local_path}. Error: {upload_error}")
                else:
                    # Log unexpected errors during head_object check
                    logging.error(f"Error checking existence of {s3_key}: {e}")


In [ ]:

#turn this into a config file
upload_parquet_files_to_s3(
    bucket_name='alpha-hmcts-de-testing-sandbox',
    local_directory='data/example-data',
    s3_prefix='de-intro-project-jb/path',
    dry_run=True
)

### Load data

In [ ]:
import s3fs

def list_parquet_files_from_s3(bucket_name: str, s3_prefix: str) -> list:
    """
    Lists all Parquet files in a given S3 prefix using s3fs.

    Parameters:
        bucket_name (str): S3 bucket name.
        s3_prefix (str): Prefix (folder path) in the bucket.

    Returns:
        list: List of full S3 paths to Parquet files.
    """
    fs = s3fs.S3FileSystem()
    s3_path = f"s3://{bucket_name}/{s3_prefix}"
    files = fs.ls(s3_path)
    return [f for f in files if f.endswith('.parquet')]

In [ ]:
from arrow_pd_parser import reader
import pandas as pd

def load_parquet_files_from_s3(bucket_name, s3_prefix):
    """
    Loads and parses Parquet files from S3 using PyArrow and a custom parser.

    Parameters:
        bucket_name (str): S3 bucket name.
        s3_prefix (str): Prefix (folder path) in the bucket.

    Returns:
        pd.DataFrame: Combined DataFrame from all Parquet files.
    """
    #from your_module import list_parquet_files_from_s3  # adjust import as needed

    parquet_files = list_parquet_files_from_s3(bucket_name, s3_prefix)
    all_dfs = []

    for file_path in parquet_files:
        df = reader.read(file_path)  # reader handles S3 paths directly
        all_dfs.append(df)

    return pd.concat(all_dfs, ignore_index=True) if all_dfs else pd.DataFrame()


In [ ]:
#load data
bucket = "alpha-hmcts-de-testing-sandbox"
prefix = "de-intro-project-jb/path"

df = load_parquet_files_from_s3(bucket, prefix)
df.head()



,User Id,First Name,Last Name,Email,Phone,Date of birth,Job Title,Source extraction date,Index
0,e09c4f4cbfEFaFd,Dawn,Trevino,clintongood@example.org,360-423-5286,1972-01-17,"Teacher, primary school",2024-02-29 12:30:10,1
1,D781D28b845Ab9D,Dale,Mcknight,clairebradshaw@example.org,9062423229,1931-01-31,"Development worker, community",2024-02-29 12:30:10,2
2,eda7EcaF87b2D80,Herbert,Bean,johnnybooker@example.org,001-149-154-0679x1617,2018-02-10,Ceramics designer,2024-02-29 12:30:10,3
3,E75ACea5D7AeC3e,Karen,Everett,wkhan@example.org,870.294.7563x20939,1938-06-14,"Civil engineer, consulting",2024-02-29 12:30:10,4
4,9C4Df1246ddf543,Angela,Shea,reginaldgarner@example.com,242.442.2978,1971-11-22,Health and safety adviser,2024-02-29 12:30:10,5
